In [123]:
from os.path import exists, join
from os import mkdir
from datetime import datetime, timedelta, date
from collections import namedtuple

import numpy as np
import pandas as pd
import scipy as sp

In [124]:
train_base_dir = '../dataset/fetures/'

# 移动滑窗划分成9个predict样本 

In [125]:
df = pd.read_csv('../dataset/train.csv', parse_dates=['record_date'], index_col=['record_date'])

In [126]:
begin = df.index.max().date() + timedelta(1)

## 训练样本划分

In [127]:
TimeSpan = namedtuple('TimeSpan', ['train_start', 'train_end', 'predict_start', 'predict_end'])

In [128]:
timespans = []

In [129]:
end = begin - timedelta(1)
for i in range(9):
    p_end = date(begin.year, begin.month-i, begin.day)
    predict_end = p_end - timedelta(1)
    predict_start = date(begin.year, begin.month-i-1, begin.day)
    train_end = predict_start - timedelta(1)
    train_start = date(predict_start.year-1, predict_start.month, predict_start.day)
    span = TimeSpan(train_start, train_end, predict_start, predict_end)
#     print(span)
    timespans.append(span)

##### 创建文件架 

In [130]:
for span in timespans:
    dir_n = str(span.predict_start.month)
    path = join(train_base_dir, dir_n)
    if not exists(path):
        mkdir(path)

In [131]:
span = timespans[0]

In [132]:
def create_train_sample(timespan):  
    train = df.loc[str(span.train_start):str(span.train_end)]    
    train_predict = df.loc[str(span.predict_start):str(span.predict_end)].reset_index()
    train_predict = train_predict.groupby('record_date')[['power_consumption']].sum()
    train_predict['predict_power_consumption'] = train_predict.power_consumption.astype(np.int64)
    train_predict.drop('power_consumption', axis=1, inplace=True)
    train_predict.index.name = 'predict_date'
    train_path = join(train_base_dir, str(span.predict_start.month), 'train.csv')
    train_predict_path = join(train_base_dir, str(span.predict_start.month), 'train_predict.csv')
    train.to_csv(train_path)
    train.info()
    train_predict.to_csv(train_predict_path)
    train_predict.info()

In [133]:
for span in timespans:
    create_train_sample(span)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532156 entries, 2015-09-01 to 2016-08-31
Data columns (total 2 columns):
user_id              532156 non-null int64
power_consumption    532156 non-null int64
dtypes: int64(2)
memory usage: 12.2 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30 entries, 2016-09-01 to 2016-09-30
Data columns (total 1 columns):
predict_power_consumption    30 non-null int64
dtypes: int64(1)
memory usage: 480.0 bytes
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532156 entries, 2015-08-01 to 2016-07-31
Data columns (total 2 columns):
user_id              532156 non-null int64
power_consumption    532156 non-null int64
dtypes: int64(2)
memory usage: 12.2 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2016-08-01 to 2016-08-31
Data columns (total 1 columns):
predict_power_consumption    31 non-null int64
dtypes: int64(1)
memory usage: 496.0 bytes
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532153 entries, 2015-

In [138]:
test = df.loc['2015/10/1':'2016/9/30']

In [139]:
index = pd.date_range('2016/10/1', '2016/10/31')
test_predict = pd.DataFrame(0, index=index, columns=['predict_power_consumption'])
test_predict.index.name = 'predict_date'

In [140]:
test.to_csv('../dataset/fetures/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532159 entries, 2015-10-01 to 2016-09-28
Data columns (total 2 columns):
user_id              532159 non-null int64
power_consumption    532159 non-null int64
dtypes: int64(2)
memory usage: 12.2 MB


In [142]:
test_predict.to_csv('../dataset/fetures/test_predict.csv')
test_predict.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2016-10-01 to 2016-10-31
Freq: D
Data columns (total 1 columns):
predict_power_consumption    31 non-null int64
dtypes: int64(1)
memory usage: 496.0 bytes
